In [1]:
from pandas import read_csv # For dataframes
from pandas import DataFrame # For dataframes
from numpy import ravel # For matrices
import matplotlib.pyplot as plt # For plotting data
import seaborn as sns # For plotting data
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV # For optimization


In [2]:
# Read ecoli dataset from the UCI ML Repository and store in
# dataframe df
df = read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/ecoli/ecoli.data',
    sep = '\s+',
    header=None)
print(df.head())

            0     1     2     3    4     5     6     7   8
0   AAT_ECOLI  0.49  0.29  0.48  0.5  0.56  0.24  0.35  cp
1  ACEA_ECOLI  0.07  0.40  0.48  0.5  0.54  0.35  0.44  cp
2  ACEK_ECOLI  0.56  0.40  0.48  0.5  0.49  0.37  0.46  cp
3  ACKA_ECOLI  0.59  0.49  0.48  0.5  0.52  0.45  0.36  cp
4   ADI_ECOLI  0.23  0.32  0.48  0.5  0.55  0.25  0.35  cp


In [3]:
# The data matrix X
X = df.iloc[:,1:-1]
# The labels
y = (df.iloc[:,-1:])
 
# Encode the labels into unique integers
encoder = LabelEncoder()
y = encoder.fit_transform(ravel(y))
 
# Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(
    X,  
    y, 
    test_size=1/3,
    random_state=0)
 
print(X_train.shape)
print(X_test.shape)

(224, 7)
(112, 7)


In [5]:
knn = KNeighborsClassifier().fit(X_train, y_train)
print('Training set score: ' + str(knn.score(X_train,y_train)))
print('Test set score: ' + str(knn.score(X_test,y_test)))

Training set score: 0.9017857142857143
Test set score: 0.8482142857142857


In [6]:
pipe = Pipeline([
('scaler', StandardScaler()),
('selector', VarianceThreshold()),
('classifier', KNeighborsClassifier())
])

In [7]:
pipe.fit(X_train, y_train)
 
print('Training set score: ' + str(pipe.score(X_train,y_train)))
print('Test set score: ' + str(pipe.score(X_test,y_test)))

Training set score: 0.8794642857142857
Test set score: 0.8392857142857143


In [8]:
parameters = {'scaler': [StandardScaler(), MinMaxScaler(),
	Normalizer(), MaxAbsScaler()],
	'selector__threshold': [0, 0.001, 0.01],
	'classifier__n_neighbors': [1, 3, 5, 7, 10],
	'classifier__p': [1, 2],
	'classifier__leaf_size': [1, 5, 10, 15]
}

In [9]:
grid = GridSearchCV(pipe, parameters, cv=2).fit(X_train, y_train)
 
print('Training set score: ' + str(grid.score(X_train, y_train)))
print('Test set score: ' + str(grid.score(X_test, y_test)))

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Training set score: 0.8928571428571429
Test set score: 0.8571428571428571


In [10]:
# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)

{'classifier__leaf_size': 1, 'classifier__n_neighbors': 7, 'classifier__p': 2, 'scaler': StandardScaler(), 'selector__threshold': 0}
Pipeline(steps=[('scaler', StandardScaler()),
                ('selector', VarianceThreshold(threshold=0)),
                ('classifier',
                 KNeighborsClassifier(leaf_size=1, n_neighbors=7))])


In [11]:
result_df = DataFrame.from_dict(grid.cv_results_, orient='columns')
print(result_df.columns)

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_classifier__leaf_size', 'param_classifier__n_neighbors',
       'param_classifier__p', 'param_scaler', 'param_selector__threshold',
       'params', 'split0_test_score', 'split1_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')


In [12]:
result_df

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.002499  5.002022e-04         0.002501    4.994869e-04   
1         0.002000  2.980232e-06         0.002499    5.006790e-04   
2         0.001996  3.576279e-06         0.002504    5.037785e-04   
3         0.001500  4.999638e-04         0.002496    5.035400e-04   
4         0.000996  3.337860e-06         0.002504    5.038977e-04   
..             ...           ...              ...             ...   
475       0.002001  5.960464e-07         0.001999    2.384186e-07   
476       0.001001  1.192093e-07         0.002000    3.576279e-07   
477       0.001001  5.960464e-07         0.002998    2.384186e-07   
478       0.002002  3.576279e-07         0.001999    7.152557e-07   
479       0.001501  4.998446e-04         0.001999    1.192093e-07   

    param_classifier__leaf_size param_classifier__n_neighbors  \
0                             1                             1   
1                             1                             1   
2                             1                             1   
3                             1                             1   
4                             1                             1   
..                          ...                           ...   
475                          15                            10   
476                          15                            10   
477                          15                            10   
478                          15                            10   
479                          15                            10   

    param_classifier__p      param_scaler param_selector__threshold  \
0                     1  StandardScaler()                         0   
1                     1  StandardScaler()                     0.001   
2                     1  StandardScaler()                      0.01   
3                     1    MinMaxScaler()                         0   
4                     1    MinMaxScaler()                     0.001   
..                  ...               ...                       ...   
475                   2      Normalizer()                     0.001   
476                   2      Normalizer()                      0.01   
477                   2    MaxAbsScaler()                         0   
478                   2    MaxAbsScaler()                     0.001   
479                   2    MaxAbsScaler()                      0.01   

                                                params  split0_test_score  \
0    {'classifier__leaf_size': 1, 'classifier__n_ne...           0.830357   
1    {'classifier__leaf_size': 1, 'classifier__n_ne...           0.830357   
2    {'classifier__leaf_size': 1, 'classifier__n_ne...           0.830357   
3    {'classifier__leaf_size': 1, 'classifier__n_ne...           0.848214   
4    {'classifier__leaf_size': 1, 'classifier__n_ne...           0.848214   
..                                                 ...                ...   
475  {'classifier__leaf_size': 15, 'classifier__n_n...           0.812500   
476  {'classifier__leaf_size': 15, 'classifier__n_n...           0.687500   
477  {'classifier__leaf_size': 15, 'classifier__n_n...           0.821429   
478  {'classifier__leaf_size': 15, 'classifier__n_n...           0.821429   
479  {'classifier__leaf_size': 15, 'classifier__n_n...           0.812500   

     split1_test_score  mean_test_score  std_test_score  rank_test_score  
0             0.767857         0.799107        0.031250              329  
1             0.767857         0.799107        0.031250              329  
2             0.767857         0.799107        0.031250              329  
3             0.741071         0.794643        0.053571              361  
4             0.741071         0.794643        0.053571              361  
..                 ...              ...             ...              ...  
475           0.839286         0.825893        0.013393               73  
476           0.7

In [13]:
sns.relplot(data=result_df,
	kind='line',
	x='param_classifier__n_neighbors',
	y='mean_test_score',
	hue='param_scaler',
	col='param_classifier__p')
plt.show()

In [14]:
sns.relplot(data=result_df,
            kind='line',
            x='param_classifier__n_neighbors',
            y='mean_test_score',
            hue='param_scaler',
            col='param_classifier__leaf_size')
plt.show()